## Google colab setup

The following cells are needed to install all dependencies we need for our tutorial on google colab.

Before executing those, make sure to **change the Hardware accelerator to GPU**. (Runtime -> change runtime type)

In [ ]:
# check if we are running in google colab
# if we are running in colab, we install conda/mamba and all dependencies we need
import sys
import os
import subprocess
from distutils.spawn import find_executable

# test if we run in google colab
in_colab = 'google.colab' in sys.modules
print(f'running in colab: {in_colab}')

# test if cuda is present
cuda_present = find_executable('nvidia-smi') is not None
print(f'cuda present: {cuda_present}')

if in_colab:
    # install condacolab to get the conda/mamba command which we need to install parallelproj
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'condacolab'])
    import condacolab
    condacolab.install()

    #-----------------------------------------------------------------------------
    #--- !!! the kernel gets restarted here !!! ----------------------------------
    #-----------------------------------------------------------------------------

    # if we we are running in colab, we install parallelproj from conda-forge
    # and set the environment variable for the parallelproj c/cuda library
    # we need to redo the check for COLAB, because the install
    # of conda on colab, restarts the kernel and deletes all variables 
   

    # install all dependencies
    subprocess.run(['mamba', 'install', 'parallelproj', 'scipy', 'matplotlib', 'nibabel', 'ipykernel', 'pytorch', 'torchmetrics'])
    
    # install cupy if cuda is present
    if cuda_present:
        subprocess.run(['mamba', 'install', 'cupy'])

    subprocess.run(['mamba', 'list'])
    subprocess.run(['git', 'clone', 'https://github.com/gschramm/parallelproj-examples.git'])

In [ ]:
import sys
import os
from distutils.spawn import find_executable

# test if we run in google colab
in_colab = 'google.colab' in sys.modules
print(f'running in colab: {in_colab}')

# test if cuda is present
cuda_present = find_executable('nvidia-smi') is not None
print(f'cuda present: {cuda_present}')

if in_colab:
    os.environ['PARALLELPROJ_C_LIB']='/usr/local/lib/libparallelproj_c.so'
    if cuda_present:
        os.environ['PARALLELPROJ_CUDA_LIB']='/usr/local/lib/libparallelproj_cuda.so'

In [ ]:
%cd parallelproj-examples/2023-MIC

---
---
---

# Minimal parallelproj demo using non-TOF projections

In [ ]:
import parallelproj
# import the numpy or cupy as array backend
if parallelproj.cupy_enabled:
    import cupy as xp
else:
    import numpy as xp

import matplotlib.pyplot as plt

In [ ]:
# setup an dummy image + image geometry
n0 = 128
n1 = 128
n2 = 25
img = xp.zeros((n0,n1,n2), dtype = xp.float32)
img[(n0//4):((3*n0//4)), (n1//4):((3*n1//4)), :] = 1
img[70:75,70:75, :] = 4

voxel_size = xp.array([3., 3., 2.]).astype(xp.float32)
img_origin = ((-xp.array(img.shape) / 2 + 0.5) * voxel_size).astype(xp.float32)

In [ ]:
# load pre-defined LOR start / end points from file
lor_coords = xp.load('lor_coordinates.npz')
xstart = lor_coords['xstart']
xend = lor_coords['xend']

In [ ]:
# non-TOF forward projection
img_fwd = xp.zeros(xstart.shape[:-1], dtype = xp.float32)

parallelproj.joseph3d_fwd(xstart.reshape(-1, 3),
                xend.reshape(-1, 3),
                img,
                img_origin, voxel_size, img_fwd)
print(type(img_fwd))
print(img_fwd.shape)
#-

In [ ]:
# non-TOF backprojection
img_back = xp.zeros(img.shape, dtype = xp.float32)

parallelproj.joseph3d_back(xstart.reshape(-1, 3),
                           xend.reshape(-1, 3), img_back,
                           img_origin, voxel_size,
                           img_fwd)
print(type(img_back))
print(img_back.shape)
#-

In [ ]:
# visualize the results
fig, ax = plt.subplots(1,3, figsize = (12,4))
ax[0].imshow(parallelproj.tonumpy(img[...,0], xp), cmap = 'Greys')
ax[1].imshow(parallelproj.tonumpy(img_fwd[...,0], xp), cmap = 'Greys')
ax[2].imshow(parallelproj.tonumpy(img_back[...,0], xp), cmap = 'Greys')
fig.tight_layout()
plt.show()